In [1]:
import cvxpy as cp
import pandas as pd
import matplotlib.pyplot as plt

from functions.retro_data.weather_data import get_weather
from functions.calculations.calculations import calculate_energy
from functions.deciding_functions.optimized import optimize_energy_flow
from functions.deciding_functions.rulebased import rule_based_energy_flow

In [2]:
# Parameter
solar_peak_power = 38  # MWp
wind_peak_power = 22   # MWp
hourly_demand = 1.6 * 39000 / (365 * 24)  # MWh
battery_capacity = 12  # MWh
initial_battery = 6.0  # MWh

In [3]:
#Funktionsaufrufe

# Wetterdaten abrufen
df_weather = get_weather()
if df_weather is None:
    print("Abbruch wegen Fehler bei der API.")

# Bedarf und Ertrag berechnen
df_energy = calculate_energy(df_weather, solar_peak_power, wind_peak_power, hourly_demand)

# Optimierungsfunktionen
result_df_rule = rule_based_energy_flow(df_energy, battery_capacity, initial_battery)
result_df_opt, total_cost = optimize_energy_flow(df_energy, battery_capacity, initial_battery)

RuntimeError: Solver für MIP nicht installiert.

In [ ]:
# Kostenberechnung
result_df_rule["costs"] = result_df_rule["grid_buy"] * 100
total_cost_rule = result_df_rule["costs"].sum()

In [ ]:
#Ausgaben 
print(f"Regelbasiert:")
print(f"Stromkosten: {total_cost_rule:.2f} €")
print("Netzbezug:", result_df_rule["grid_buy"].sum())
print("Gesamtladung Batterie:", result_df_rule["battery_charge"].sum())
print("Gesamtentladung Batterie:", result_df_rule["battery_discharge"].sum())

print(f"Optimiert:")
print(f"Stromkosten: {total_cost:.2f} €")
print("Netzbezug:", result_df_opt["grid_buy"].sum())
print("Gesamtladung Batterie:", result_df_opt["battery_charge"].sum())
print("Gesamtentladung Batterie:", result_df_opt["battery_discharge"].sum())

In [ ]:
#Als .csv speichern
result_df_rule.to_csv("data\regelbasierter_energiefluss.csv", index=False)
result_df_opt.to_csv("data\optimierter_energiefluss.csv", index=False)

In [ ]:
#Plots erstellen

# CSV-Dateien laden
df_opt = pd.read_csv("data\optimierter_energiefluss.csv")
df_regel = pd.read_csv("data\regelbasierter_energiefluss.csv")

# Zeitspalte parsen
df_opt["datetime"] = pd.to_datetime(df_opt["datetime"])
df_regel["datetime"] = pd.to_datetime(df_regel["datetime"])

# Filter: nur die ersten beiden Tage
start_time = df_opt["datetime"].min()
start_time = start_time + pd.Timedelta(days=1)
end_time = start_time + pd.Timedelta(days=3)

df_opt = df_opt[(df_opt["datetime"] >= start_time) & (df_opt["datetime"] < end_time)]
df_regel = df_regel[(df_regel["datetime"] >= start_time) & (df_regel["datetime"] < end_time)]

# Schriftgrößen definieren
label_fontsize = 20
tick_fontsize = 20
legend_fontsize = 20

# Plot erstellen
fig, axs = plt.subplots(4, 1, figsize=(14, 12), sharex=True)

# Netzbezug
axs[0].plot(df_opt["datetime"], df_opt["grid_buy"], label="optimized", linewidth=2.5)
axs[0].plot(df_regel["datetime"], df_regel["grid_buy"], label="rulebased", color="black", linestyle="--", linewidth=2)
axs[0].set_ylabel("Grid electricity [MWh]", fontsize=label_fontsize)
axs[0].legend(fontsize=legend_fontsize)
axs[0].tick_params(axis='both', labelsize=tick_fontsize)

# Curtailment vs. Feed-in
axs[1].plot(df_opt["datetime"], df_opt["curtailment"], label="optimized", color="orange", linewidth=2.5)
axs[1].plot(df_regel["datetime"], df_regel["grid_feed_in"], label="rulebased", color="black", linestyle="--", linewidth=2)
axs[1].set_ylabel("Feed-in [MWh]", fontsize=label_fontsize)
axs[1].legend(fontsize=legend_fontsize)
axs[1].tick_params(axis='both', labelsize=tick_fontsize)

# Batteriezustand
axs[2].plot(df_opt["datetime"], df_opt["battery_state"], label="optimized", color="green", linewidth=2.5)
axs[2].plot(df_regel["datetime"], df_regel["battery_state"], label="rulebased", color="black", linestyle="--", linewidth=2)
axs[2].set_ylabel("SOC [MWh]", fontsize=label_fontsize)
axs[2].legend(fontsize=legend_fontsize)
axs[2].tick_params(axis='both', labelsize=tick_fontsize)

# Erzeugung und Verbrauch
axs[3].plot(df_opt["datetime"], df_opt["total_energy_production"], label="Energy generation", color="purple", linewidth=2.5)
axs[3].plot(df_opt["datetime"], df_opt["energy_demand"], label="Energy demand", linestyle="--", color="black", linewidth=2)
axs[3].set_ylabel("Energy quantity [MWh]", fontsize=label_fontsize)
axs[3].legend(fontsize=legend_fontsize)
axs[3].tick_params(axis='both', labelsize=tick_fontsize)

# Gemeinsame X-Achse
plt.xlabel("Date/time", fontsize=label_fontsize)
plt.tight_layout()
plt.show()

RuntimeError: Solver für MIP nicht installiert.